In [3]:
import requests
import hashlib

class VTIsMalicious:

    # Your VirusTotal API Key
    API_KEY = "b55521d99967e7b1c7234130ec227616e6b16d59bdbdced12389fb5fbddb13be"
    # VirusTotal API endpoint for file hash report
    URL = "https://www.virustotal.com/api/v3/files/"

    def __init__(self):
        self.file = "/home/rosn/Documents/python/projectR2.0/datasets/benign/1721907289.pdf"
    
    def file_to_hash(self, filepath: str, algorithm: str = "sha256") -> str:
        """
        Compute the hash of a file using the specified algorithm.

        :param filepath: Path to the file to hash.
        :param algorithm: Hashing algorithm to use (default is 'sha256').
                          Options: 'md5', 'sha1', 'sha256'.
        :return: The computed hash as a string.
        """
        try:
            # Validate the hashing algorithm
            if algorithm not in hashlib.algorithms_available:
                raise ValueError(f"Unsupported algorithm '{algorithm}'. Choose from: {', '.join(hashlib.algorithms_available)}")
            
            # Initialize the hasher
            hasher = hashlib.new(algorithm)
            
            # Read the file in chunks to avoid memory issues with large files
            with open(filepath, "rb") as f:
                for chunk in iter(lambda: f.read(4096), b""):
                    hasher.update(chunk)
            
            # Return the hex digest of the hash
            return hasher.hexdigest()
        except FileNotFoundError:
            return f"Error: File not found at {filepath}"
        except Exception as e:
            return f"Error: {e}"

    def check_file_hash(self) -> str:
        # Compute the hash of the file
        file_hash = self.file_to_hash(self.file)
        
        if "Error" in file_hash:
            return file_hash  # Return error message
        
        headers = {"x-apikey": self.API_KEY}
        try:
            response = requests.get(self.URL + file_hash, headers=headers)

            if response.status_code == 200:
                data = response.json()
                if "data" in data:
                    attributes = data["data"]["attributes"]
                    malicious_count = attributes.get("last_analysis_stats", {}).get("malicious", 0)
                    total_engines = sum(attributes.get("last_analysis_stats", {}).values())
                    
                    result = f"File Hash: {file_hash}\nMalicious Detections: {malicious_count}/{total_engines}\n"
                    if malicious_count > 0:
                        result += "Result: Malicious"
                    else:
                        result += "Result: Clean"
                    return result
                else:
                    return f"No data found for hash: {file_hash}"
            else:
                return f"Error: {response.status_code} - {response.text}"
        except Exception as e:
            return f"An error occurred: {e}"


In [4]:
scanner = VTIsMalicious()
print(scanner.check_file_hash())

Error: 404 - {"error": {"code": "NotFoundError", "message": "File \"aeea6236c3cb8b69fe55d9a04066d36500a12232836c2de5ca8e058af4cee120\" not found"}}
